In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

<a id="notebook_id"></a>
# Priority queues

A priority queue is a queue where the elements are key-value pairs. The keys do not need to be unique but they must be comparable (using less than, equal to, and greater than). A key is interpreted as the *priority* of the element. In a priority queue, elements with higher priority are served before elements with lower priority (some descriptions of priority queues serve elements with lower priority first). A value is simply any value that we want to store in the queue.

Examples of priority queues in real life include:

* a hospital emergency room where patients are seen by doctors in order of the severity of the patients' condition
* some entertainment venues allow VIPs to move to the front of the queue
* CPU scheduling algorithms found in operating systems may use a priority queue to schedule processes
* many network applications use priority queues to manage bandwidth

Some descriptions of priority queues do not use key-value pairs; such priority queues require that the elements be comparable.

## Interface

There is no consensus on the name of the operations provided by a priority queue and most descriptions and implementations do not use the queue operation names enqueue and dequeue (for example, the standard implementations in C++, Python, and Java all use different names for their priority queue operations).

We will use the following interface for a priority queue:

* `size` : returns the number of key-value pairs in the queue
* `isEmpty` : returns true if the queue is empty, false otherwise
* `insert(K priority, V value)` : inserts a value with the specified priority
* `max` : returns the key-value pair having the greatest priority
* `removeMax` : removes and returns the key-value pair having the greatest priority

In [ ]:
package ca.queensu.cs.cisc235.pq;

/**
 * A priority queue is a queue where the elements are key-value pairs. The keys
 * do not need to be unique but they must be comparable. A key is interpreted as
 * the priority of the element. In a priority queue, elements with higher
 * priority are served before elements with lower priority.
 *
 * @param <K> the key (priority) type
 * @param <V> the value type
 */
public interface PriorityQueue<K extends Comparable<K>, V> {

    /**
     * A priority queue entry (key-value pair). The key represents the priority of
     * the value.
     *
     * @param <K> the key (priority) type
     * @param <V> the value type
     */
    public interface Entry<K, V> {

        /**
         * Gets the priority (key) of this entry.
         * 
         * @return the priority (key) of this entry
         */
        public K getPriority();

        /**
         * Gets the value of this entry.
         * 
         * @return the value of this entry
         */
        public V getValue();

        /**
         * Sets the value of this entry returning the overwritten value.
         * 
         * @param value the new value of this entry
         * @return the overwritten value of this entry
         */
        public V setValue(V value);
    }

    /**
     * Returns the number of key-value pairs in this queue.
     * 
     * @return the number of key-value pairs in this queue
     */
    public int size();

    /**
     * Returns true if this queue is empty, false otherwise.
     * 
     * @return true if this queue is empty, false otherwise
     */
    public boolean isEmpty();

    /**
     * Adds the specified value to this queue with the specified priority. Values
     * with higher priority are served by the queue before values with lower
     * priority.
     * 
     * @param priority the priority of the specified value
     * @param value    a value
     */
    public void insert(K priority, V value);

    /**
     * Returns the key-value pair having the highest priority.
     * 
     * @return the key-value pair having the highest priority
     */
    public Entry<K, V> max();

    /**
     * Removes and returns the key-value pair having the highest priority.
     * 
     * @return the key-value pair having the highest priority
     */
    public Entry<K, V> removeMax();

}


The `PriorityQueue` interface has a nested interface `Entry` that represents a key-value pair. We use a nested interface to keep the queue and its entry interface grouped together.

Most entries are not very interesting; they consist of a reference to a key that cannot be null and a reference to a value. An implementation of a simple entry is shown below:

In [ ]:
package ca.queensu.cs.cisc235.pq;

import ca.queensu.cs.cisc235.pq.PriorityQueue.Entry;

public class SimpleEntry<K extends Comparable<K>, V> implements Entry<K, V> {

    private K priority;
    private V value;

    /**
     * Initializes this entry to have the specified priority and value.
     * 
     * @param priority a priority
     * @param value    a value
     * @throws NullPointerException if the priority is null
     */
    public SimpleEntry(K priority, V value) {
        if (priority == null) {
            throw new NullPointerException("null priority");
        }
        this.priority = priority;
        this.value = value;
    }

    @Override
    public K getPriority() {
        return this.priority;
    }

    @Override
    public V getValue() {
        return this.value;
    }

    @Override
    public V setValue(V value) {
        V old = this.value;
        this.value = value;
        return old;
    }

    /**
     * Returns a string representation of this map entry. This implementation
     * returns the string representation of this entry's key followed by the equals
     * character ("=") followed by the string representation of this entry's value.
     * 
     * @return a string representation of this entry
     */
    @Override
    public String toString() {
        return this.priority.toString() + "=" + this.value.toString();
    }

}

By this point in the course, the reader should be able to design and implement a simple priority queue class. For example, one way to implement a priority queue is to use an array of unsorted key-value pairs; getting and removing the highest priority key-value pair uses exhaustive search over the array:

In [ ]:
package ca.queensu.cs.cisc235.pq;

import java.util.Arrays;

public class FixedArrayPriorityQueue<K extends Comparable<K>, V> implements PriorityQueue<K, V> {

    /**
     * The default capacity of a queue.
     */
    public static int DEFAULT_CAPACITY = 16;
    
    private Object[] arr;
    
    // the number of elements in the queue
    private int size;
    
    /**
     * Initializes an empty queue having the default capacity.
     */
    public FixedArrayPriorityQueue() {
        this(DEFAULT_CAPACITY);
    }
    
    /**
     * Initializes an empty queue having the specified capacity.
     */
    public FixedArrayPriorityQueue(int capacity) {
        if (capacity < 0) {
            throw new IllegalArgumentException("negative capacity: " + capacity);
        }
        this.arr = new Object[capacity];
        this.size = 0;
    }
    
    @Override
    public int size() {
        return this.size;
    }

    @Override
    public boolean isEmpty() {
        return this.size() == 0;
    }

    @Override
    public void insert(K priority, V value) {
        // simply adds to the end of the queue
        if (this.size() == this.arr.length) {
            throw new RuntimeException("queue full");
        }
        // next line will throw NullPointerException if priority is null
        Entry<K, V> e = new SimpleEntry<>(priority, value);
        this.arr[this.size] = e;
        this.size++;
    }

    private int maxIndex() {
        // exhaustive search of a non-empty queue
        Entry<K, V> e = (Entry<K, V>) this.arr[0];
        K pmax = e.getPriority();
        int imax = 0;
        for (int i = 1; i < this.size; i++) {
            e = (Entry<K, V>) this.arr[i];
            K pi = e.getPriority();
            int cmp = pi.compareTo(pmax);
            if (cmp > 0) {
                pmax = pi;
                imax = i;
            }
        }
        return imax;
    }
    
    @Override
    public Entry<K, V> max() {
        if (this.isEmpty()) {
            throw new RuntimeException("empty queue");
        }
        Entry<K, V> e = (Entry<K, V>) this.arr[this.maxIndex()];
        return e;
    }

    @Override
    public Entry<K, V> removeMax() {
        if (this.isEmpty()) {
            throw new RuntimeException("empty queue");
        }
        int imax = this.maxIndex();
        Entry<K, V> result = (Entry<K, V>) this.arr[imax];
        
        // shift elements forward one position
        for (int i = imax; i < this.size; i++) {
            this.arr[i] = this.arr[i + 1];
        }
        // null out last element
        this.arr[this.size - 1] = null;
        this.size--;
        return result;
    }
    
    @Override
    public String toString() {
        return Arrays.toString(this.arr);
    }

    
    public static void main(String[] args) {
        PriorityQueue<Integer, String> q = new FixedArrayPriorityQueue<>();
        
        q.insert(5, "five");
        System.out.println(q);
        
        q.insert(1, "one");
        System.out.println(q);
        
        q.insert(3, "three");
        System.out.println(q);
        
        Entry<Integer, String> e = q.removeMax();
        System.out.println("removed: " + e);
        System.out.println(q);
        
        e = q.max();
        System.out.println("max: " + e);
        System.out.println(q);
    }
}


Run all of the previous code cells to compile the `FixedArrayPriorityQueue` class and then run the next cell to run the `main` method.

In [ ]:
ca.queensu.cs.cisc235.pq.FixedArrayPriorityQueue.main(null);

**Exercise 1** What are the complexities of `insert`, `max`, and `removeMax` for an unsorted array-based priority queue?

**Exercise 2** What are the complexities of `insert`, `max`, and `removeMax` for a sorted array-based priority queue where the key-value pairs are sorted from highest to lowest priority?

**Exercise 3** Design and implement a priority queue that uses a stack for storage of its key-value pairs.

**Exercise 4** Design and implement a priority queue that uses a conventional queue for storage of its key-value pairs.